In [7]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
SEED = 1013
np.random.seed(SEED)
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 
from stance_utils import *
#from utils import *
#from parameters import *
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout,Concatenate,Dense, Embedding, LSTM, SpatialDropout1D, Flatten, GRU, Bidirectional, Conv1D, Input,MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
stemmer = PorterStemmer()
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stopwords_english = stopwords.words('english')
from sklearn.preprocessing import LabelEncoder

In [8]:
classes = {'FAVOR': np.array([1, 0, 0]), 'NONE': np.array([0, 1, 0]), 'AGAINST': np.array([0, 0, 1])}
classes_ = np.array(['FAVOR', 'NONE', 'AGAINST'])

In [9]:
train_data_file = '/data/parush/stance_mohammed/train.txt'
test_data_file = '/data/parush/stance_mohammed/test.txt'
TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]


# train_data_file = '/data/parush/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file = '/data/parush/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']


# train_data_file = '/data/parush/Data_MPCHI/train.txt'
# test_data_file = '/data/parush/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']


In [10]:
def train_and_test():
    
    sentence_maxlen = 0
    x_train = []
    y_train = []

    
    with open(train_data_file, 'r') as trainfile:
        for line in trainfile:
            
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            #if line[0].strip() != 'ID' and line[1].strip() == t:
            if line[0].strip() != 'ID':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_train.append(tweet)
                y_train.append(classes[line[3].strip()])
                               
    
    x_test = []
    y_test = []
    with open(test_data_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            #if line[0] != 'ID' and line[1] == t:
            if line[0] != 'ID':
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_test.append(tweet)
                y_test.append(classes[line[3].strip()])


    
    return x_train, y_train, x_test, y_test, sentence_maxlen

In [11]:
x_train, y_train, x_test, y_test, sentence_maxlen = train_and_test()

In [12]:
vocabulary = build_vocab(x_train)
vocab_size = len(vocabulary)
print("Total words in vocab are",vocab_size)

Total words in vocab are 6497


In [13]:

embeddings_weights = get_embeddings('wikipedia',100,vocabulary)



In [14]:
for i in range(len(x_train)):
    tweet_tensor = tweet_to_tensor(x_train[i], vocabulary)
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_train[i] = tweet_tensor
for i in range(len(x_test)):
    tweet_tensor = tweet_to_tensor(x_test[i], vocabulary)
    if len(tweet_tensor) < sentence_maxlen:
        diff = sentence_maxlen - len(tweet_tensor)
        n_pad = [0]*diff
        tweet_tensor = tweet_tensor + n_pad
    x_test[i] = tweet_tensor

In [15]:
x_train = np.array(x_train)

x_test = np.array(x_test)

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

2914

In [18]:
num_classes = 3
def biLSTM():
    model = Sequential()
    model.add(Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights]))
    model.add(Dropout(0.2))
    model.add(LSTM(64,return_sequences=True,dropout=0.3))
    model.add(Bidirectional(LSTM(64,dropout=0.3)))
    #model.add(Flatten())
    #add a dropout here
    model.add(Dropout(0.5))
    model.add(Dense(num_classes,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

def biLSTMCNN():
    inputs = Input(shape=(sentence_maxlen,))
    embedded_inputs = Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights])(inputs)
    embedded_inputs = Dropout(0.2)(embedded_inputs)
    lstm = Bidirectional(LSTM(64,return_sequences=True,dropout=0.3))(embedded_inputs)
    convs = []
    for each_filter_size in [3,4,5]:
        #print(rnn.shape)
        each_conv = Conv1D(100, each_filter_size, activation='relu')(lstm)
        each_conv = MaxPooling1D(sentence_maxlen-each_filter_size+1)(each_conv)
        each_conv = Flatten()(each_conv)
        #print(each_conv.shape)
        convs.append(each_conv)
        
    output = Concatenate()(convs)
    output = Dropout(0.5)(output)
    output = (Dense(3,activation='softmax'))(output)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) 
    return model

def biGRU():
    model = Sequential()
    model.add(Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights]))
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(64,return_sequences=True,dropout=0.3)))
    model.add(Bidirectional(GRU(64,dropout=0.3)))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

def biGRUCNN():
    inputs = Input(shape=(sentence_maxlen,))
    embedded_inputs = Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights])(inputs)
    embedded_inputs = Dropout(0.2)(embedded_inputs)
    rnn = Bidirectional(GRU(64,return_sequences=True,dropout=0.3))(embedded_inputs)
    convs = []
    for each_filter_size in [3,4,5]:
        #print(rnn.shape)
        each_conv = Conv1D(100, each_filter_size, activation='relu')(rnn)
        each_conv = MaxPooling1D(sentence_maxlen-each_filter_size+1)(each_conv)
        each_conv = Flatten()(each_conv)
        #print(each_conv.shape)
        convs.append(each_conv)
        
    output = Concatenate()(convs)
    output = Dropout(0.5)(output)
    output = (Dense(3,activation='softmax'))(output)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])    
    
    return model

In [19]:
model = biLSTMCNN()
print(model.summary())


2021-08-30 18:13:49.349371: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/cuda/cuda-11.2/lib64:/opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib:/opt/rh/devtoolset-8/root/usr/lib64/dyninst:/opt/rh/devtoolset-8/root/usr/lib/dyninst:/opt/rh/devtoolset-8/root/usr/lib64:/opt/rh/devtoolset-8/root/usr/lib:/usr/include/slurm/
2021-08-30 18:13:49.349421: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-30 18:13:49.349459: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gaivi.cse.usf.edu): /proc/driver/nvidia/version does not exist
2021-08-30 18:13:49.349760: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 100)      649700      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 20, 100)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20, 128)      84480       dropout[0][0]                    
______________________________________________________________________________________________

In [20]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
cvscores = []

for train, val in kfold.split(x_train, classes_[y_train.argmax(1)]):
    model.fit(x_train[train], y_train[train], epochs = 50, batch_size = 16, verbose=1)
    scores = model.evaluate(x_train[val], y_train[val], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

y_pred = np.round(model.predict(x_test))

print("For: ",classification_report(y_test, y_pred, digits=4))


2021-08-30 18:13:55.797218: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-30 18:13:55.797852: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2399910000 Hz


Epoch 1/50
146/146 [==============================] - 5s 15ms/step - loss: 1.0290 - accuracy: 0.4869
Epoch 2/50
146/146 [==============================] - 2s 15ms/step - loss: 0.9152 - accuracy: 0.5701
Epoch 3/50
146/146 [==============================] - 2s 15ms/step - loss: 0.7903 - accuracy: 0.6461
Epoch 4/50
146/146 [==============================] - 2s 15ms/step - loss: 0.6275 - accuracy: 0.7490
Epoch 5/50
146/146 [==============================] - 2s 15ms/step - loss: 0.4669 - accuracy: 0.8164
Epoch 6/50
146/146 [==============================] - 2s 15ms/step - loss: 0.3628 - accuracy: 0.8541
Epoch 7/50
146/146 [==============================] - 2s 15ms/step - loss: 0.2644 - accuracy: 0.8958
Epoch 8/50
146/146 [==============================] - 2s 15ms/step - loss: 0.2043 - accuracy: 0.9266
Epoch 9/50
146/146 [==============================] - 2s 15ms/step - loss: 0.1526 - accuracy: 0.9421
Epoch 10/50
146/146 [==============================] - 2s 15ms/step - loss: 0.1535 - accura

146/146 [==============================] - 2s 15ms/step - loss: 0.0083 - accuracy: 0.9974
Epoch 12/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0094 - accuracy: 0.9979
Epoch 13/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0069 - accuracy: 0.9987
Epoch 14/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0057 - accuracy: 0.9983
Epoch 15/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0050 - accuracy: 0.9979
Epoch 16/50
146/146 [==============================] - 2s 16ms/step - loss: 0.0086 - accuracy: 0.9970
Epoch 17/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0074 - accuracy: 0.9979
Epoch 18/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0034 - accuracy: 0.9987
Epoch 19/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0101 - accuracy: 0.9991
Epoch 20/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0012 - accuracy:

146/146 [==============================] - 3s 19ms/step - loss: 0.0018 - accuracy: 0.9996
Epoch 42/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0024 - accuracy: 0.9991
Epoch 43/50
146/146 [==============================] - 3s 20ms/step - loss: 0.0051 - accuracy: 0.9983
Epoch 44/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0139 - accuracy: 0.9966
Epoch 45/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0141 - accuracy: 0.9979
Epoch 46/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0019 - accuracy: 0.9991
Epoch 47/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0018 - accuracy: 0.9991
Epoch 48/50
146/146 [==============================] - 3s 19ms/step - loss: 0.0033 - accuracy: 0.9987
Epoch 49/50
146/146 [==============================] - 3s 18ms/step - loss: 0.0028 - accuracy: 0.9983
Epoch 50/50
146/146 [==============================] - 2s 15ms/step - loss: 0.0021 - accuracy:

/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
For:                precision    recall  f1-score   support

           0     0.4413    0.5197    0.4773       304
           1     0.3815    0.4130    0.3967       230
           2     0.7314    0.6476    0.6869       715

   micro avg     0.5774    0.5733    0.5753      1249
   macro avg     0.5181    0.5268    0.5203      1249
weighted avg     0.5964    0.5733    0.5825      1249
 samples avg     0.5733    0.5733    0.5733      1249